<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mapping-Rehabilitation-Centers---Pediatric" data-toc-modified-id="Mapping-Rehabilitation-Centers---Pediatric-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mapping Rehabilitation Centers - Pediatric</a></span><ul class="toc-item"><li><span><a href="#Loading-population-of-states" data-toc-modified-id="Loading-population-of-states-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading population of states</a></span></li><li><span><a href="#Loading-dataset-centers" data-toc-modified-id="Loading-dataset-centers-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Loading dataset centers</a></span></li><li><span><a href="#Fixing-latitude-and-longitude" data-toc-modified-id="Fixing-latitude-and-longitude-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Fixing latitude and longitude</a></span></li><li><span><a href="#Merge-to-create-number-of-beds-per-population" data-toc-modified-id="Merge-to-create-number-of-beds-per-population-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Merge to create number of beds per population</a></span></li></ul></li><li><span><a href="#Map" data-toc-modified-id="Map-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Map</a></span><ul class="toc-item"><li><span><a href="#States" data-toc-modified-id="States-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>States</a></span></li><li><span><a href="#Census-Divisions" data-toc-modified-id="Census-Divisions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Census Divisions</a></span></li><li><span><a href="#Regions" data-toc-modified-id="Regions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Regions</a></span></li><li><span><a href="#Map-for-manuscript" data-toc-modified-id="Map-for-manuscript-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Map for manuscript</a></span><ul class="toc-item"><li><span><a href="#Converting-latitude-and-longitude-of-facilities-to-the-correct-projection" data-toc-modified-id="Converting-latitude-and-longitude-of-facilities-to-the-correct-projection-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Converting latitude and longitude of facilities to the correct projection</a></span></li></ul></li><li><span><a href="#Test-manuscript-map" data-toc-modified-id="Test-manuscript-map-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Test manuscript map</a></span></li><li><span><a href="#Manuscript-maps" data-toc-modified-id="Manuscript-maps-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Manuscript maps</a></span><ul class="toc-item"><li><span><a href="#State---to-be-included-in-the-manuscript" data-toc-modified-id="State---to-be-included-in-the-manuscript-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>State - <font color="red">to be included in the manuscript</font></a></span><ul class="toc-item"><li><span><a href="#Test:-beds/children" data-toc-modified-id="Test:-beds/children-2.6.1.1"><span class="toc-item-num">2.6.1.1&nbsp;&nbsp;</span>Test: beds/children</a></span></li></ul></li><li><span><a href="#Division-with-beds/children" data-toc-modified-id="Division-with-beds/children-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>Division with beds/children</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
%matplotlib inline
user = 'UF'

import datetime as dt
import pandas as pd
import sys
import csv
import glob
import ipywidgets #as widgets
import itertools
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import socket


import requests
import logging
import time

from matplotlib import *
from matplotlib.dates import YearLocator, MonthLocator, DateFormatter
from matplotlib.dates import *
#style.use('ggplot')
from numpy.random import randn
#from scipy import statss
from pandas import  DataFrame, Series
import statsmodels.api as sm

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots

sns.set()

import warnings
warnings.filterwarnings('ignore')

## display entire text columns
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_columns", None)

today = dt.datetime.today().strftime("%y%m%d")
print(today)


if user == 'UF':

    %run py_addresses.py

elif user =='MM':
    ## Here I define 
    #'addressdata', which is the folder where the data is located
    # 'data_tools', which is where the data_tools are located
    # 'address' is not used anywhere and mostly deprecated
    print(user)
    
    
else:
    print("Define the paths for `address`, `addressdata`, and `data_tools`")
    
    
from IPython.core.display import HTML
HTML("<style>.container { width:80% !important; }</style>")


# Mapping Rehabilitation Centers - Pediatric

## Loading population of states

In [ ]:
## https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-detail.html
df_pop = pd.read_csv('csv_data/sc-est2019-agesex-civ.csv', dtype=object)

In [ ]:
df_pop.head()

In [ ]:
list_ages = np.arange(0,15)
list_ages = [str(x) for x in list_ages]
print(list_ages)

In [ ]:
## Selecting only 0-14 year-olds
df_pop = df_pop[df_pop['AGE'].astype(str).isin(list_ages)]

## Selecting only states
df_pop = df_pop[df_pop['NAME'] != 'United States']

# groupby STATE and 2019
df_pop['POPEST2019_CIV'] = df_pop['POPEST2019_CIV'].astype(int)
df_pop_group = df_pop.groupby('NAME')['POPEST2019_CIV'].sum().reset_index(name='Children')
df_pop_group

In [ ]:
dict_us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
df_pop_group['STATE'] = df_pop_group['NAME'].map(dict_us_state_abbrev)
df_pop_group

## Loading dataset centers 

In [ ]:
df_rehab = pd.read_csv('csv_data/pac_facilities_dataset_230330.csv', dtype=object)
df_rehab['STATE'] = df_rehab['STATE'].apply(lambda x: str(x).replace(" ", ""))

df_rehab['STATUS'] = df_rehab['STATUS'].astype(str)
df_rehab['STATUS'].value_counts()


##################################################################
### REMOVING CENTERS WITHOUT LATITUDE/Longitude
print("\n")
print("="*120)
print("Centers without location (latitude/longitude) ---> We'll keep only the ones with location to map")

df_rehab[['latitude', 'longitude']] = df_rehab[['latitude', 'longitude']].astype(float)
df_without_location = df_rehab[df_rehab['latitude'].isna()]
#display(df_without_location[['STATE','FAC_NAME_CMS','ST_ADR_NS','FullAddress','STATUS_desc','latitude', 'longitude']])
display(df_without_location[['STATE','FAC_NAME','ST_ADR','STATUS','latitude', 'longitude']])

## Keeping the ones with location
df_rehab = df_rehab[~df_rehab['latitude'].isna()]

print("="*120)
print("\n")
########################################################################################


#df_rehab.head(2)

In [ ]:
df_rehab['STATUS'].value_counts()

In [ ]:
# df_rehab['STATUS_DESC'] = 'POST-ACUTE CARE FACILITY THAT ACCEPTS ADOLESCENT PATIENTS'

# df_rehab['STATUS_DESC'] = np.where(df_rehab['STATUS'] == '3.0', 
#                                    'POST-ACUTE CARE FACILITY THAT ACCEPTS PEDIATRIC PATIENTS ON A CASE-BY-CASE BASIS', 
#                                    df_rehab['STATUS_DESC'])
                    
# df_rehab['STATUS_DESC'] = np.where((df_rehab['STATUS'] == '1.0') & 
#                                     (df_rehab['FREE'] == '0'), 
#                                    'FACILITY WITH EMBEDDED PEDIATRIC POST-ACUTE CARE UNIT',
#                                    df_rehab['STATUS_DESC'])

# df_rehab['STATUS_DESC'] = np.where((df_rehab['STATUS'] == '1.0') & (df_rehab['FREE'] == '1'), 
#                                    'FREE-STANDING PEDIATRIC POST-ACUTE CARE FACILITY',
#                                    df_rehab['STATUS_DESC'])  

df_rehab['STATUS_desc'] = 'Post-acute care facility accepting adolescent patients'

df_rehab['STATUS_desc'] = np.where(df_rehab['STATUS'] == '3.0', 
                                   'Post-acute care facility accepting pediatric patients on a case-by-case basis', 
                                   df_rehab['STATUS_desc'])
                    
df_rehab['STATUS_desc'] = np.where((df_rehab['STATUS'] == '1.0') & 
                                    (df_rehab['FREE'] == '0'), 
                                   'Facility with embedded pediatric post-acute care unit',
                                   df_rehab['STATUS_desc'])

df_rehab['STATUS_desc'] = np.where((df_rehab['STATUS'] == '1.0') & 
                                   (df_rehab['FREE'] == '1'), 
                                   'Free-standing pediatric post-acute care facility',
                                   df_rehab['STATUS_desc'])  

In [ ]:
df_rehab['STATUS_desc'].value_counts()

In [ ]:
# ## Rank to sort labels in the map
# dict_rank_pac = {'Free-standing pediatric PAC facility':1,
#                  'PAC facility with embedded pediatric unit':2,
#                  'PAC facility accepting pediatric patients':3,
#                  'PAC facility accepting adolescent patients':4}
# df_rehab['STATUS_rank'] = df_rehab['STATUS_desc'].map(dict_rank_pac)

## Rank to sort labels in the map
dict_rank_pac = {'Free-standing pediatric post-acute care facility':1,
                 'Facility with embedded pediatric post-acute care unit':2,
                 'Post-acute care facility accepting pediatric patients on a case-by-case basis':3,
                 'Post-acute care facility accepting adolescent patients':4}
df_rehab['STATUS_rank'] = df_rehab['STATUS_desc'].map(dict_rank_pac)

In [ ]:
df_rehab['INPT_BEDS_map'] = df_rehab['INPT_BEDS']
df_rehab['INPT_BEDS_map'] = np.where(df_rehab['INPT_BEDS_map']=='60/66', 60, df_rehab['INPT_BEDS_map'])
df_rehab['INPT_BEDS_map'] = df_rehab['INPT_BEDS_map'].astype(float)
#df_rehab['INPT_BEDS_map'].astype(float).describe()

## Annotation on circle for number of beds
df_rehab['INPT_BEDS_map_comment'] = np.where(df_rehab['INPT_BEDS_map'].isna(),
                                            'No information', 
                                             df_rehab['INPT_BEDS_map'])

## size of circle on map
df_rehab['INPT_BEDS_map_circle'] = np.where(df_rehab['INPT_BEDS_map'].isna(),
                                            10, 
                                             df_rehab['INPT_BEDS_map'])

In [ ]:
dict_trach = {'0': 'No',
              '1': 'Yes',
              '2':'Case-by-case',
              '3': 'Unknown'
             }
df_rehab['TRACH_desc'] = df_rehab['TRACH'].map(dict_trach).replace(np.nan, 'Unknown')
print(df_rehab['TRACH_desc'].value_counts(), '\n')



dict_vent = {'0': 'No',
             '1': 'Yes',
             '2':'Case-by-case',
            '3': 'Unknown'
            }
df_rehab['VENT_desc'] = df_rehab['VENT'].map(dict_vent).replace(np.nan, 'Unknown')
print(df_rehab['VENT_desc'].value_counts(), '\n')


dict_pn = {'0': 'No',
             '1': 'Yes',
             '2':'Case-by-case',
            '3': 'Unknown'
            }
df_rehab['PN_desc'] = df_rehab['PN'].map(dict_vent).replace(np.nan, 'Unknown')
print(df_rehab['PN_desc'].value_counts(), '\n')

## Fixing latitude and longitude

In [ ]:
# df_rehab['latitude'] = np.where(df_rehab['FAC_NAME_NS']=='TAR RIVER',
#                                35.62954122471422,
#                                df_rehab['latitude'])

# df_rehab['longitude'] = np.where(df_rehab['FAC_NAME_NS']=='TAR RIVER',
#                                -77.4241428079374,
#                                df_rehab['longitude'])

# df_rehab[df_rehab['FAC_NAME_NS']=='TAR RIVER'][['FullAddress', 'latitude', 'longitude']]

## Merge to create number of beds per population

In [ ]:
list_status = ['Facility with embedded pediatric post-acute care unit','Free-standing pediatric post-acute care facility']
                
df_rehab_group = df_rehab[df_rehab['STATUS_desc'].isin(list_status)].groupby('STATE')['INPT_BEDS_map'].sum().reset_index(name='State_beds')

In [ ]:
### Number accepting pediatric
display(df_rehab['STATUS'].value_counts())

df_rehab_ped_centers = df_rehab[df_rehab['STATUS']=='1.0']
print(len(df_rehab_ped_centers))

## Number per State
df_rehab_ped_centers_group = df_rehab_ped_centers.groupby('STATE').size().reset_index(name='Pediatric_centers')

In [ ]:
## Fixing white spaces on the name of states
df_rehab_group['STATE'] = df_rehab_group['STATE'].apply(lambda x: str(x).replace(" ", ""))

df_merged = df_rehab_group.merge(df_pop_group, on='STATE', how='outer').replace(np.nan, 0)
df_merged = df_merged.merge(df_rehab_ped_centers_group, on='STATE', how='outer').replace(np.nan, 0)
df_merged['beds_per_100k'] = round(1e5*df_merged['State_beds']/df_merged['Children'],2)
df_merged['Children_M'] = round(df_merged['Children']/1e6,3)
#df_merged.sorted()

# Map

## States

In [ ]:
### Loading geojson file for states
import json
with open(data_tools+"geojson/geojson_states") as geofile:
    geojson_layer_state = json.load(geofile)
    
geojson_layer_state["features"][0]["properties"]

In [ ]:
df_rehab['STATUS_rank'] = df_rehab['STATUS_desc'].map(dict_rank_pac)

df_rehab = df_rehab.sort_values('STATUS_rank')

In [ ]:
plot_map = 'Y'

if plot_map == 'Y':
    ######## Plotting choropleth
 
    fig = px.choropleth_mapbox(df_merged, #dataframe with numbers and zip code
                               geojson=geojson_layer_state, # the geojson file we created with zip codes
                               locations='STATE', #the column of df with locations
                               color='Children_M',#'beds_per_100k',#'Children',  #the column of df with numbers
                               color_continuous_scale='Greys',#"OrRd", #color scale
                               featureidkey="properties.STUSPS",#"properties.POSTCODE", #the key in the geojson file with zipcode. See sect 1.4
                               mapbox_style="carto-positron",# "white-bg",#"carto-positron", # style of background map
                               zoom=3, #initial zoom, only if we want
                               center = {"lat": 37.09, "lon": -95.71}, #center of the map, only if we want
                               opacity=0.6, #opacity of the choropleth
#                                hover_data={'STATE','Pediatric_centers','beds_per_100k', 'Children_M'}, #What will be shown in the hover box (besides location) 
                                hover_data={'STATE','Pediatric_centers','Children_M'}, #What will be shown in the hover box (besides location) 
                               labels={'Pediatric_centers':'Centers accepting pediatric patients', 
#                                        'beds_per_100k':'Beds per 100k children in State',
                                       'Children_M': 'Population under 18 years (millions)',
                                       'STATE': 'State',#labels in the hover box
                                       })

    
    
    color_options = ['Blue', 'DarkOrange', 'DarkRed', 'Black']
 
    i=0
    list_status_full = ['Free-standing pediatric post-acute care facility',
                 'Facility with embedded pediatric post-acute care unit',
                 'Post-acute care facility accepting pediatric patients on a case-by-case basis',
                 'Post-acute care facility accepting adolescent patients']
    for status in list_status_full:
        print(i+1, status)
        sizepoint=4
                    

        df_centers = df_rehab[df_rehab['STATUS_desc'] == status]
        
        if status in ['Post-acute care facility accepting pediatric patients on a case-by-case basis',
                 'Post-acute care facility accepting adolescent patients']:
            df_centers['INPT_BEDS_map_circle'] = 8
            df_centers['INPT_BEDS_map_comment'] = '-'
            
        df_centers['INPT_BEDS_map_comment'] = df_centers['INPT_BEDS_map_comment'].astype(str)

        site_lat = df_centers['latitude']
        site_lon = df_centers['longitude']
        site_name = ('<b>'+df_centers['FAC_NAME']+" ("+df_centers['STATE']+')</b><br>'+str('Beds: ')+df_centers['INPT_BEDS_map_comment'].astype(str)+'<br>'+str('Status: ')+df_centers['STATUS_desc'].astype(str)+
                    '<br>'+str('Tracheostomy: ')++df_centers['TRACH_desc'] +
                    '<br>'+str('Mechanical Ventilation: ')++df_centers['VENT_desc'] +
                    '<br>'+str('Parenteral Nutrition: ')++df_centers['PN_desc'])
        site_number = df_centers['INPT_BEDS_map_circle'].apply(lambda x: np.sqrt(x))

        fig.add_trace(go.Scattermapbox(
                lat=site_lat,
                lon=site_lon,
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=site_number*sizepoint,
                    color=color_options[i],#site_icu,
                    #colorscale='Black',
                    opacity=0.65,
                    #showscale=True,
                    #colorscale=[(0, "DimGrey"), (0.15, "slategrey"), (0.5, "darkslategrey"), (1, "black")]
                ),
                 text=site_name,
                 hoverinfo='text',
                 name=status,
            ))

        fig.update_traces(showlegend=True)
        i+=1

    fig.update_layout(title={
                    'text': "Post-Acute Care Centers",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                      legend=dict(x=-.1, y=0.8), 
                      coloraxis_colorbar=dict(title="Population under 18 years (millions)"),
                      coloraxis=dict(cmax=11, cmin=0),
                      margin={"r":0,"t":0,"l":0,"b":0})

    
    
#    fig.show()
    plot(fig, filename="webpage_maps/rehab_centers_states.html")
#    plot(fig, filename="webpage_maps/rehab_centers_beds_10k.html")
    
# #     plotly.io.write_image(fig, "figures/fig1a_hosps_peds_BW.pdf", #format=pdf,
# #                    width=1000, height=600, validate=True)
    
else:
    print("To show the map, change variable to 'Y' ")

## Census Divisions

In [ ]:
import geopandas

shp_file = geopandas.read_file(data_tools+'geojson/cb_2018_us_division_500k/cb_2018_us_division_500k.shp')
shp_file.to_file(data_tools+'geojson/geojson_divisions', driver='GeoJSON')

In [ ]:
### Loading geojson file for states
import json
with open(data_tools+"geojson/geojson_divisions") as geofile:
    geojson_layer_division = json.load(geofile)
    
geojson_layer_division["features"][0]["properties"]


In [ ]:
df_state_to_regions = pd.read_excel(data_tools+"geojson/state_FIPS_codes.xlsx", dtype=object)
dict_state_to_divisions = df_state_to_regions.set_index("state_cd")['div_name'].to_dict()
#dict_state_to_divisions 

In [ ]:
df_merged['Division'] = df_merged['STATE'].map(dict_state_to_divisions)
df_merged_divisions = df_merged.groupby('Division')[['State_beds', 'Children', 'Pediatric_centers']].sum().reset_index()
df_merged_divisions = df_merged_divisions.rename(columns={'State_beds':'Division_beds'})
df_merged_divisions['Beds_per_100k'] = round(1e5*df_merged_divisions['Division_beds']/df_merged_divisions['Children'],2)
df_merged_divisions

df_merged_divisions['Children_M'] = round(df_merged_divisions['Children']/1e6,3)
df_merged_divisions


In [ ]:
plot_map = 'N'

if plot_map == 'Y':
    ######## Plotting choropleth
 
    fig = px.choropleth_mapbox(df_merged_divisions, #dataframe with numbers and zip code
                               geojson=geojson_layer_division, # the geojson file we created with zip codes
                               locations='Division', #the column of df with locations
                               color='Children_M',#'Children',  #the column of df with numbers
                               color_continuous_scale='Greys',#"OrRd", #color scale
                               featureidkey="properties.NAME",#"properties.POSTCODE", #the key in the geojson file with zipcode. See sect 1.4
                               mapbox_style="carto-positron",# "white-bg",#"carto-positron", # style of background map
                               zoom=3, #initial zoom, only if we want
                               center = {"lat": 37.09, "lon": -95.71}, #center of the map, only if we want
                               opacity=0.6, #opacity of the choropleth
                               #hover_data={'Children_M','Division','Pediatric_centers','Beds_per_100k'}, #What will be shown in the hover box (besides location) 
                               hover_data={'Children_M','Division','Pediatric_centers'},
                               labels={'Pediatric_centers':'Centers accepting pediatric pts', 
                                       #'Beds_per_100k':'Beds per 100k children in Division',
                                       'Children_M':'Population under 18 years (millions)',
                                       'Division': 'Division',#labels in the hover box
                                       })

    
    
    color_options = ['Blue', 'DarkOrange', 'DarkRed', 'Black']
 
    i=0
    list_status_full = ['Free-standing pediatric post-acute care facility',
                 'Facility with embedded pediatric post-acute care unit',
                 'Post-acute care facility accepting pediatric patients on a case-by-case basis',
                 'Post-acute care facility accepting adolescent patients']
    for status in list_status_full:
        print(i+1, status)
        sizepoint=4
                    

        df_centers = df_rehab[df_rehab['STATUS_desc'] == status]
        
        if status in ['Post-acute care facility accepting pediatric patients on a case-by-case basis',
                 'Post-acute care facility accepting adolescent patients']:
            df_centers['INPT_BEDS_map_circle'] = 8
            df_centers['INPT_BEDS_map_comment'] = '-'
            
        df_centers['INPT_BEDS_map_comment'] = df_centers['INPT_BEDS_map_comment'].astype(str)

        site_lat = df_centers['latitude']
        site_lon = df_centers['longitude']
        site_name = ('<b>'+df_centers['FAC_NAME']+" ("+df_centers['STATE']+')</b><br>'+str('Beds: ')+df_centers['INPT_BEDS_map_comment'].astype(str)+'<br>'+str('Status: ')+df_centers['STATUS_desc'].astype(str)+
                    '<br>'+str('Tracheostomy: ')++df_centers['TRACH_desc'] +
                    '<br>'+str('Mechanical Ventilation: ')++df_centers['VENT_desc'] +
                    '<br>'+str('Parenteral Nutrition: ')++df_centers['PN_desc'])
        site_number = df_centers['INPT_BEDS_map_circle'].apply(lambda x: np.sqrt(x))

        fig.add_trace(go.Scattermapbox(
                lat=site_lat,
                lon=site_lon,
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=site_number*sizepoint,
                    color=color_options[i],#site_icu,
                    #colorscale='Black',
                    opacity=0.65,
                    #showscale=True,
                    #colorscale=[(0, "DimGrey"), (0.15, "slategrey"), (0.5, "darkslategrey"), (1, "black")]
                ),
                 text=site_name,
                 hoverinfo='text',
                 name=status,
            ))

        fig.update_traces(showlegend=True)
        i+=1

    fig.update_layout(title={
                    'text': "Post-Acute Care Centers",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                      legend=dict(x=-.1, y=0.8), 
                      coloraxis_colorbar=dict(title="Population under 18 years (millions)"),
                      margin={"r":0,"t":0,"l":0,"b":0})

    
    
#    fig.show()
    plot(fig, filename="webpage_maps/rehab_centers_divisions.html")
#    plot(fig, filename="webpage_maps/rehab_centers_beds_10k.html")
    
# #     plotly.io.write_image(fig, "figures/fig1a_hosps_peds_BW.pdf", #format=pdf,
# #                    width=1000, height=600, validate=True)
    
else:
    print("To show the map, change variable to 'Y' ")

## Regions

In [ ]:
### Loading geojson file for states
import json
with open(data_tools+"geojson/geojson_regions") as geofile:
    geojson_layer_region = json.load(geofile)
    
geojson_layer_region["features"][0]["properties"]


In [ ]:
df_state_to_regions = pd.read_excel(data_tools+"geojson/state_FIPS_codes.xlsx", dtype=object)
dict_state_to_regions = df_state_to_regions.set_index("state_cd")['reg_name'].to_dict()
#dict_state_to_regions 

In [ ]:
df_merged['Region'] = df_merged['STATE'].map(dict_state_to_regions)
df_merged_regions = df_merged.groupby('Region')[['State_beds', 'Children', 'Pediatric_centers']].sum().reset_index()
df_merged_regions = df_merged_regions.rename(columns={'State_beds':'Region_beds'})
df_merged_regions['Beds_per_100k'] = round(1e5*df_merged_regions['Region_beds']/df_merged_regions['Children'],2)
df_merged_regions

df_merged_regions['Children_M'] = round(df_merged_regions['Children']/1e6,3)
df_merged_regions

In [ ]:
plot_map = 'N'

if plot_map == 'Y':
    ######## Plotting choropleth
 
    fig = px.choropleth_mapbox(df_merged_regions, #dataframe with numbers and zip code
                               geojson=geojson_layer_region, # the geojson file we created with zip codes
                               locations='Region', #the column of df with locations
                               color='Children_M',#'Children',  #the column of df with numbers
                               color_continuous_scale='Greys',#"OrRd", #color scale
                               featureidkey="properties.NAME",#"properties.POSTCODE", #the key in the geojson file with zipcode. See sect 1.4
                               mapbox_style="carto-positron",# "white-bg",#"carto-positron", # style of background map
                               zoom=3, #initial zoom, only if we want
                               center = {"lat": 37.09, "lon": -95.71}, #center of the map, only if we want
                               opacity=0.6, #opacity of the choropleth
                               hover_data={'Region','Pediatric_centers','Children_M'}, #What will be shown in the hover box (besides location) 
                               labels={'Pediatric_centers':'Centers accepting pediatric pts', 
                                       #'Beds_per_100k':'Beds per 100k children in State',
                                       'Children_M':'Population under 18 years (millions)',
                                       'Region': 'Region',#labels in the hover box
                                       })

    
    
    color_options = ['Blue', 'DarkOrange', 'DarkRed', 'Black']
 
    i=0
    list_status_full = ['Free-standing pediatric post-acute care facility',
                 'Facility with embedded pediatric post-acute care unit',
                 'Post-acute care facility accepting pediatric patients on a case-by-case basis',
                 'Post-acute care facility accepting adolescent patients']
    for status in list_status_full:
        print(i+1, status)
        sizepoint=4
                    

        df_centers = df_rehab[df_rehab['STATUS_desc'] == status]
        
        if status in ['Post-acute care facility accepting pediatric patients on a case-by-case basis',
                 'Post-acute care facility accepting adolescent patients']:
            df_centers['INPT_BEDS_map_circle'] = 8
            df_centers['INPT_BEDS_map_comment'] = '-'
            
        df_centers['INPT_BEDS_map_comment'] = df_centers['INPT_BEDS_map_comment'].astype(str)

        site_lat = df_centers['latitude']
        site_lon = df_centers['longitude']
        site_name = ('<b>'+df_centers['FAC_NAME']+" ("+df_centers['STATE']+')</b><br>'+str('Beds: ')+df_centers['INPT_BEDS_map_comment'].astype(str)+'<br>'+str('Status: ')+df_centers['STATUS_desc'].astype(str)+
                    '<br>'+str('Tracheostomy: ')++df_centers['TRACH_desc'] +
                    '<br>'+str('Mechanical Ventilation: ')++df_centers['VENT_desc'] +
                    '<br>'+str('Parenteral Nutrition: ')++df_centers['PN_desc'])
        site_number = df_centers['INPT_BEDS_map_circle'].apply(lambda x: np.sqrt(x))

        fig.add_trace(go.Scattermapbox(
                lat=site_lat,
                lon=site_lon,
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=site_number*sizepoint,
                    color=color_options[i],#site_icu,
                    #colorscale='Black',
                    opacity=0.65,
                    #showscale=True,
                    #colorscale=[(0, "DimGrey"), (0.15, "slategrey"), (0.5, "darkslategrey"), (1, "black")]
                ),
                 text=site_name,
                 hoverinfo='text',
                 name=status,
            ))

        fig.update_traces(showlegend=True)
        i+=1

    fig.update_layout(title={
                    'text': "Post-Acute Care Centers",
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                      legend=dict(x=-.1, y=0.8), 
                      coloraxis_colorbar=dict(title="Population under 18 years (millions)"),
                      margin={"r":0,"t":0,"l":0,"b":0})

    
    
#    fig.show()
    plot(fig, filename="webpage_maps/rehab_centers_population.html")
#    plot(fig, filename="webpage_maps/rehab_centers_beds_10k.html")
    
# #     plotly.io.write_image(fig, "figures/fig1a_hosps_peds_BW.pdf", #format=pdf,
# #                    width=1000, height=600, validate=True)
    
else:
    print("To show the map, change variable to 'Y' ")

## Map for manuscript

In [ ]:
import geopandas as gpd
data_tools = "/Users/ufranca/Documents/Dropbox/projects_coding/data_tools/"

if user == 'UF':
#     path_geojson = data_tools+"geojson/us-states.json"
    path_geojson = data_tools+"geojson/geojson_states"
else:
    path_geojson = 'json_files\geojson_states'
    
geodf = gpd.read_file(path_geojson)
   
#display(geodf.head())
geodf = geodf.to_crs({'init': 'epsg:4326'})
geodf['geometry'] = geodf['geometry'].to_crs(epsg=3395)

geodf = geodf[(geodf['STUSPS'] != 'HI') & 
              (geodf['STUSPS'] != 'AK') &
              (geodf['STUSPS'] != 'PR') & 
              (geodf['STUSPS'] != 'MP') & 
              (geodf['STUSPS'] != 'GU') & 
              (geodf['STUSPS'] != 'AS') & 
              (geodf['STUSPS'] != 'VI')]

geodf.to_file(data_tools+"geojson/geojson_states_epsg3395", driver = "GeoJSON")
    
# print(geodf['STUSPS'].unique())
# print(geodf['NAME'].unique())
# # with open(path_geojson) as f:
# #     Mass = json.load(f)
    
path_geojson = data_tools+"geojson/geojson_states_epsg3395"
us2 = gpd.read_file(path_geojson)
#us2 = us2.rename(columns={'name':'NAME'})
# #Mass2#['GEOID']

In [ ]:
# import geopandas as gpd
# data_tools = "/Users/ufranca/Documents/Dropbox/projects_coding/data_tools/"

# if user == 'UF':
# #     path_geojson = data_tools+"geojson/us-states.json"
#     path_geojson = data_tools+"geojson/geojson_states"
# else:
#     path_geojson = 'json_files\geojson_states'
    
# geodf = gpd.read_file(path_geojson)
   
# #display(geodf.head())
# geodf = geodf.to_crs({'init': 'epsg:4326'})
# geodf['geometry'] = geodf['geometry'].to_crs(epsg=3395)

# geodf = geodf[(geodf['STUSPS'] != 'HI') & 
#               (geodf['STUSPS'] != 'AK') &
#               (geodf['STUSPS'] != 'PR') & 
#               (geodf['STUSPS'] != 'MP') & 
#               (geodf['STUSPS'] != 'GU') & 
#               (geodf['STUSPS'] != 'AS') & 
#               (geodf['STUSPS'] != 'VI')]

# geodf.to_file(data_tools+"geojson/geojson_states_epsg3395", driver = "GeoJSON")
    
# # print(geodf['STUSPS'].unique())
# # print(geodf['NAME'].unique())
# # # with open(path_geojson) as f:
# # #     Mass = json.load(f)
    
path_geojson = data_tools+"geojson/geojson_states_epsg3395"
us2 = gpd.read_file(path_geojson)
#us2 = us2.rename(columns={'name':'NAME'})
# #Mass2#['GEOID']

In [ ]:
us_map = us2.merge(df_merged, on='NAME', how='outer')
#us_map

In [ ]:
# fig, ax = plt.subplots(figsize=(15,12))
# #gdf.plot(aspect=1)
# us_map.plot(column= us_map['Children_M'], legend=True, ax=ax, #color='Children'
#             cmap='Greys', 
#            edgecolor='darkgrey', alpha=0.7, linewidth=0.5, aspect=1)
# #plt.scatter(df_rehab['longitude'].astype(float), df_rehab['latitude'].astype(float), c='Red', s=120, marker='o')

# # ## Loop through each flight plotting line depicting flight between source and destination
# # for slat,dlat, slon, dlon in zip(df_allstates_stacked_test['Latitude_Pt'], df_allstates_stacked_test["latitude"], 
# #                                               df_allstates_stacked_test['Longitude_Pt'], df_allstates_stacked_test["longitude"]):
# #     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

# #plt.title('(b) Suburban')
# # plt.xlim(-180, -65.55)#(-130, -65.55)
# # plt.ylim(24, 75)#(24, 55)
# plt.xticks([])
# plt.yticks([])

In [ ]:
#us_map

### Converting latitude and longitude of facilities to the correct projection

In [ ]:
## Converting latitude and longitude of facilities to the correct projection

df_allstates_stacked_hosp = df_rehab[['latitude','longitude','STATE', 'FAC_NAME', 'ST_ADR','STATUS_desc','INPT_BEDS_map_comment','INPT_BEDS_map_circle']]
df_allstates_stacked_hosp[['latitude','longitude']] = df_allstates_stacked_hosp[['latitude','longitude']].astype(np.float64)

##################################################################
### REMOVING CENTERS WITHOUT LATITUDE/Longitude
print("\n")
print("="*120)
print("Centers without location (latitude/longitude) ---> We'll keep only the ones with location to map")

df_without_location = df_allstates_stacked_hosp[df_allstates_stacked_hosp['latitude'].isna()]
display(df_without_location)

## Keeping the ones with location
df_allstates_stacked_hosp = df_allstates_stacked_hosp[~df_allstates_stacked_hosp['latitude'].isna()]

print("="*120)
print("\n")
########################################################################################

df_allstates_stacked_hosp['col_merge'] = 'CM'+df_allstates_stacked_hosp.index.astype(str)

gdf_allstates_stacked_hosp = gpd.GeoDataFrame(df_allstates_stacked_hosp, 
                                             geometry=gpd.points_from_xy(df_allstates_stacked_hosp['longitude'], df_allstates_stacked_hosp['latitude']),
                                             crs="EPSG:4326")
gdf_allstates_stacked_hosp['geometry'] = gdf_allstates_stacked_hosp['geometry'].to_crs(epsg=3395)
#display(gdf_allstates_stacked_hosp[['geometry','latitude','longitude']][200:])

#gdf_allstates_stacked_hosp['geometry'] = gdf_allstates_stacked_hosp['geometry'].to_crs(epsg=3395)
#display(df_allstates_stacked_hosp)
# print(len(gdf_allstates_stacked_hosp))
# display(gdf_allstates_stacked_hosp.sample(5))
#gdf_allstates_stacked_hosp['latitude'].value_counts()

#gdf_allstates_stacked_hosp['geometry'] = gdf_allstates_stacked_hosp['geometry'].to_crs(epsg=3395)

# display(gdf_allstates_stacked_hosp.sample(5))


# df_allstates_stacked_pts = df_allstates_stacked_test[['Latitude_Pt','Longitude_Pt','State_Pt', 'ZIP']]
# df_allstates_stacked_pts[['Latitude_Pt','Longitude_Pt']] = df_allstates_stacked_pts[['Latitude_Pt','Longitude_Pt']].astype(np.float64)
# df_allstates_stacked_pts['col_merge'] = 'CM'+df_allstates_stacked_pts.index.astype(str)
# gdf_allstates_stacked_pts = gpd.GeoDataFrame(df_allstates_stacked_pts, 
#                                              geometry=gpd.points_from_xy(df_allstates_stacked_pts['Longitude_Pt'], df_allstates_stacked_pts['Latitude_Pt']),
#                                              crs="EPSG:4326")

# print(len(gdf_allstates_stacked_pts))
# # display(gdf_allstates_stacked_pts.sample(5))

# gdf_allstates_stacked_hosp['geometry'] = gdf_allstates_stacked_hosp['geometry'].to_crs(epsg=3395)
# # display(gdf_allstates_stacked_pts.sample(5))

# #display(gdf_allstates_stacked_hosp) 
# gdf_allstates_stacked_hosp = gdf_allstates_stacked_hosp.reset_index(drop=True)
# # #print(gdf_allstates_stacked_hosp.sample(10).reset_index(drop=True))
# gdf_allstates_stacked_hosp
gdf_allstates_stacked_hosp['longitude_2'] = gdf_allstates_stacked_hosp['geometry'].x
gdf_allstates_stacked_hosp['latitude_2'] = gdf_allstates_stacked_hosp['geometry'].y


#gdf_allstates_stacked_hosp 

# #gdf_allstates_stacked_hosp['geometry'] = gdf_allstates_stacked_hosp['geometry'].to_crs(epsg=3395)
# gdf_allstates_stacked_pts['Longitude_Pt_2'] = gdf_allstates_stacked_pts['geometry'].x
# gdf_allstates_stacked_pts['Latitude_Pt_2'] = gdf_allstates_stacked_pts['geometry'].y
# gdf_allstates_stacked_pts.sample(10)

# gdf_allstates_stacked = gdf_allstates_stacked_hosp.merge(gdf_allstates_stacked_pts, on='col_merge', how='right')

# gdf_allstates_stacked.sample(5)


## Test manuscript map

In [ ]:
## see https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas
params = {'legend.fontsize': 15,
          'legend.handlelength': 2}
plt.rcParams.update(params)

fig, ax = plt.subplots(figsize=(15, 15))
gmap = us_map.plot(column= us_map['Children_M'], #color='Children'
            cmap='Blues', ax=ax,  
           edgecolor='darkblue', alpha=0.7, linewidth=0.5, aspect=1, 
           legend=True, legend_kwds={'shrink': 0.5, 'label':'Population under 18 years (millions)'})
#gmap.set_label('Pediatric population')
#plt.legend(loc=2, prop={'size': 6})

sns.scatterplot(data=gdf_allstates_stacked_hosp, x='longitude_2', y='latitude_2', 
                #hue='STATUS_desc', palette = ['Blue', 'DarkOrange', 'DarkRed', 'Black'])
                style='STATUS_desc', color='Black')#,size='INPT_BEDS_map_circle')
                #c='Black', s=120, marker='o', 
                #data=gdf_allstates_stacked_hosp)

# ## Loop through each flight plotting line depicting flight between source and destination
# for slat,dlat, slon, dlon in zip(gdf_allstates_stacked['Latitude_Pt_2'], gdf_allstates_stacked["latitude_2"], 
#                                               gdf_allstates_stacked['Longitude_Pt_2'], gdf_allstates_stacked["longitude_2"]):
#     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

#plt.title('(b) Suburban')
#plt.xlim(-2.0e7, -0.72e7)#(-130, -65.55)
#plt.ylim(0.27e7, 1.16e7)#(24, 55)
plt.xticks([])
plt.yticks([])
plt.xlabel("")
plt.ylabel("")

#plt.savefig('figures/overall_distances_withAK_mercartor.pdf', bbox_inches="tight")
#plt.savefig('figures/pac_with_AK_HI.pdf', bbox_inches="tight")

## Manuscript maps

### State - <font color="red">to be included in the manuscript</font>

In [ ]:
us_map[['STATE','beds_per_100k']]
#us_map = us_map[us_map['STATE'] != 'DC']

In [ ]:
## see https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas
params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
plt.rcParams.update(params)

fig, ax = plt.subplots(figsize=(10, 10))
gmap = us_map.plot(column= us_map['Children_M'], #color='Children'
            cmap='Blues', ax=ax,  facecolor='white',
           edgecolor='darkblue', alpha=0.7, linewidth=0.5, aspect=1, 
           legend=True, 
            legend_kwds={'shrink': 1.0, 'label':'Population under 18 years (millions)', 
                         'location':'bottom', 'pad':0.02})
#gmap.set_label('Pediatric population')
#plt.legend(loc=2, prop={'size': 6})
ax.set_facecolor("white")

gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp[~gdf_allstates_stacked_hosp['STATE'].isin(['HI', 'AK'])]

list_status = ['Free-standing pediatric post-acute care facility',
                 'Facility with embedded pediatric post-acute care unit']
gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp2[gdf_allstates_stacked_hosp2['STATUS_desc'].isin(list_status)]

sns.scatterplot(data=gdf_allstates_stacked_hosp2, x='longitude_2', y='latitude_2', 
                #hue='STATUS_desc', palette = ['Blue', 'DarkOrange', 'DarkRed', 'Black'])
                style='STATUS_desc', color='Black', alpha=0.75)
                #c='Black', s=120, marker='o', 
                #data=gdf_allstates_stacked_hosp)

# ## Loop through each flight plotting line depicting flight between source and destination
# for slat,dlat, slon, dlon in zip(gdf_allstates_stacked['Latitude_Pt_2'], gdf_allstates_stacked["latitude_2"], 
#                                               gdf_allstates_stacked['Longitude_Pt_2'], gdf_allstates_stacked["longitude_2"]):
#     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

#plt.title('(b) Suburban')
#plt.xlim(-2.0e7, -0.72e7)#(-130, -65.55)
#plt.ylim(0.27e7, 1.16e7)#(24, 55)
plt.xticks([])
plt.yticks([])
plt.xlabel("")
plt.ylabel("")

#legend = plt.legend(frameon = 1)

legend = plt.legend(bbox_to_anchor=(0.75, 0.34), title='Type of facility', frameon=1)
frame = legend.get_frame()
frame.set_color('white')
#ax.legend(handles=[])

#plt.savefig('figures/overall_distances_withAK_mercartor.pdf', bbox_inches="tight")
plt.savefig('figures/pac_continental_state_pop_color.pdf', bbox_inches="tight")

In [ ]:
## see https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas
params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
plt.rcParams.update(params)

fig, ax = plt.subplots(figsize=(10, 10))
gmap = us_map.plot(column= us_map['Children_M'], #color='Children'
            cmap='Greys', ax=ax,  facecolor='white',
           edgecolor='darkgrey', alpha=0.65, linewidth=0.5, aspect=1, 
           legend=True, 
            legend_kwds={'shrink': 1.0, 'label':'Population under 18 years (millions)', 
                         'location':'bottom', 'pad':0.02})
#gmap.set_label('Pediatric population')
#plt.legend(loc=2, prop={'size': 6})
ax.set_facecolor("white")



gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp[~gdf_allstates_stacked_hosp['STATE'].isin(['HI', 'AK'])]



list_status = ['Free-standing pediatric post-acute care facility',
                 'Facility with embedded pediatric post-acute care unit',]
gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp2[gdf_allstates_stacked_hosp2['STATUS_desc'].isin(list_status)]

sns.scatterplot(data=gdf_allstates_stacked_hosp2, x='longitude_2', y='latitude_2', 
                #hue='STATUS_desc', palette = ['Blue', 'DarkOrange', 'DarkRed', 'Black'])
                style='STATUS_desc', color='Black', alpha=0.75)
                #c='Black', s=120, marker='o', 
                #data=gdf_allstates_stacked_hosp)

# ## Loop through each flight plotting line depicting flight between source and destination
# for slat,dlat, slon, dlon in zip(gdf_allstates_stacked['Latitude_Pt_2'], gdf_allstates_stacked["latitude_2"], 
#                                               gdf_allstates_stacked['Longitude_Pt_2'], gdf_allstates_stacked["longitude_2"]):
#     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

#plt.title('(b) Suburban')
#plt.xlim(-2.0e7, -0.72e7)#(-130, -65.55)
#plt.ylim(0.27e7, 1.16e7)#(24, 55)
plt.xticks([])
plt.yticks([])
plt.xlabel("")
plt.ylabel("")

#legend = plt.legend(frameon = 1)

legend = plt.legend(bbox_to_anchor=(0.75, 0.34), title='Type of facility', frameon=1)
frame = legend.get_frame()
frame.set_color('white')
#ax.legend(handles=[])

#plt.savefig('figures/overall_distances_withAK_mercartor.pdf', bbox_inches="tight")
#plt.savefig('figures/pac_continental_BW.pdf', bbox_inches="tight")
plt.savefig('figures/pac_continental_state_pop_BW.pdf', bbox_inches="tight")

#### Test: beds/children

In [ ]:
# ## see https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas
# params = {'legend.fontsize': 12,
#           'legend.handlelength': 2}
# plt.rcParams.update(params)

# fig, ax = plt.subplots(figsize=(10, 10))
# gmap = us_map.plot(column= us_map['beds_per_100k'], #color='Children'
#             cmap='Greys', ax=ax,  facecolor='white',
#            edgecolor='darkgrey', alpha=0.65, linewidth=0.5, aspect=1, 
#            legend=True, 
#             legend_kwds={'shrink': 1.0, 'label':'Beds per 100k childrens', 
#                          'location':'bottom', 'pad':0.02})
# #gmap.set_label('Pediatric population')
# #plt.legend(loc=2, prop={'size': 6})
# ax.set_facecolor("white")



# # gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp[~gdf_allstates_stacked_hosp['STATE'].isin(['HI', 'AK'])]

# # list_status = ['PAC facility with embedded pediatric unit','Free-standing pediatric PAC facility']
# # gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp2[gdf_allstates_stacked_hosp2['STATUS_desc'].isin(list_status)]

# # sns.scatterplot(data=gdf_allstates_stacked_hosp2, x='longitude_2', y='latitude_2', 
# #                 #hue='STATUS_desc', palette = ['Blue', 'DarkOrange', 'DarkRed', 'Black'])
# #                 style='STATUS_desc', color='Black', alpha=0.75)
# #                 #c='Black', s=120, marker='o', 
# #                 #data=gdf_allstates_stacked_hosp)

# # ## Loop through each flight plotting line depicting flight between source and destination
# # for slat,dlat, slon, dlon in zip(gdf_allstates_stacked['Latitude_Pt_2'], gdf_allstates_stacked["latitude_2"], 
# #                                               gdf_allstates_stacked['Longitude_Pt_2'], gdf_allstates_stacked["longitude_2"]):
# #     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

# #plt.title('(b) Suburban')
# #plt.xlim(-2.0e7, -0.72e7)#(-130, -65.55)
# #plt.ylim(0.27e7, 1.16e7)#(24, 55)
# plt.xticks([])
# plt.yticks([])
# plt.xlabel("")
# plt.ylabel("")

# #legend = plt.legend(frameon = 1)

# # legend = plt.legend(bbox_to_anchor=(0.75, 0.34), title='Type of facility', frameon=1)
# # frame = legend.get_frame()
# # frame.set_color('white')
# #ax.legend(handles=[])

# #plt.savefig('figures/overall_distances_withAK_mercartor.pdf', bbox_inches="tight")
# #plt.savefig('figures/pac_states_BW.pdf', bbox_inches="tight")
# #plt.savefig('figures/pac_states_BW_with DC.pdf', bbox_inches="tight")

### Division with beds/children

In [ ]:
import geopandas as gpd
data_tools = "/Users/ufranca/Documents/Dropbox/projects_coding/data_tools/"

if user == 'UF':
#     path_geojson = data_tools+"geojson/us-states.json"
    path_geojson = data_tools+"geojson/geojson_divisions"
else:
    path_geojson = 'json_files\geojson_divisions'
    
geodf = gpd.read_file(path_geojson)
   
#display(geodf.head())
geodf = geodf.to_crs({'init': 'epsg:4326'})
geodf['geometry'] = geodf['geometry'].to_crs(epsg=3395)

# geodf = geodf[(geodf['STUSPS'] != 'HI') & 
#               (geodf['STUSPS'] != 'AK') &
#               (geodf['STUSPS'] != 'PR') & 
#               (geodf['STUSPS'] != 'MP') & 
#               (geodf['STUSPS'] != 'GU') & 
#               (geodf['STUSPS'] != 'AS') & 
#               (geodf['STUSPS'] != 'VI')]

geodf.to_file(data_tools+"geojson/geojson_divisions_epsg3395", driver = "GeoJSON")
    
# print(geodf['STUSPS'].unique())
# print(geodf['NAME'].unique())
# # with open(path_geojson) as f:
# #     Mass = json.load(f)
    
path_geojson = data_tools+"geojson/geojson_divisions_epsg3395"
us2 = gpd.read_file(path_geojson)
us2 = us2.rename(columns={'NAME':'Division'})
# #Mass2#['GEOID']

In [ ]:
#us2['NAME']

In [ ]:
us_map_div = us2.merge(df_merged_divisions, on='Division', how='outer')
#us_map_div

In [ ]:
# fig, ax = plt.subplots(figsize=(15,12))
# #gdf.plot(aspect=1)
# us_map_div.plot(column= us_map_div['Children_M'], legend=True, ax=ax, #color='Children'
#             cmap='Greys', 
#            edgecolor='darkgrey', alpha=0.7, linewidth=0.5, aspect=1)
# #plt.scatter(df_rehab['longitude'].astype(float), df_rehab['latitude'].astype(float), c='Red', s=120, marker='o')

# # ## Loop through each flight plotting line depicting flight between source and destination
# # for slat,dlat, slon, dlon in zip(df_allstates_stacked_test['Latitude_Pt'], df_allstates_stacked_test["latitude"], 
# #                                               df_allstates_stacked_test['Longitude_Pt'], df_allstates_stacked_test["longitude"]):
# #     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

# #plt.title('(b) Suburban')
# plt.xlim(-1.4e7, -0.73e7)#(-130, -65.55)
# plt.ylim(0.26e7, 0.65e7)#(24, 55)
# plt.xticks([])
# plt.yticks([])

In [ ]:
## see https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas
params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
plt.rcParams.update(params)

fig, ax = plt.subplots(figsize=(10, 10))
gmap = us_map_div.plot(column= us_map_div['Beds_per_100k'], #color='Children'
            cmap='Blues', ax=ax,  facecolor='white',
           edgecolor='darkblue', alpha=0.7, linewidth=0.5, aspect=1, 
           legend=True, 
            legend_kwds={'shrink': 1.0, 'label':'Beds per 100k children', 
                         'location':'bottom', 'pad':0.02})
#gmap.set_label('Pediatric population')
#plt.legend(loc=2, prop={'size': 6})
ax.set_facecolor("white")

gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp[~gdf_allstates_stacked_hosp['STATE'].isin(['HI', 'AK'])]

list_status = ['Free-standing pediatric post-acute care facility',
                 'Facility with embedded pediatric post-acute care unit',]
gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp2[gdf_allstates_stacked_hosp2['STATUS_desc'].isin(list_status)]

sns.scatterplot(data=gdf_allstates_stacked_hosp2, x='longitude_2', y='latitude_2', 
                #hue='STATUS_desc', palette = ['Blue', 'DarkOrange', 'DarkRed', 'Black'])
                style='STATUS_desc', color='Black', alpha=0.75)
                #c='Black', s=120, marker='o', 
                #data=gdf_allstates_stacked_hosp)

# ## Loop through each flight plotting line depicting flight between source and destination
# for slat,dlat, slon, dlon in zip(gdf_allstates_stacked['Latitude_Pt_2'], gdf_allstates_stacked["latitude_2"], 
#                                               gdf_allstates_stacked['Longitude_Pt_2'], gdf_allstates_stacked["longitude_2"]):
#     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

#plt.title('(b) Suburban')
plt.xlim(-1.4e7, -0.73e7)#(-130, -65.55)
plt.ylim(0.26e7, 0.65e7)#(24, 55)
plt.xticks([])
plt.yticks([])
plt.xlabel("")
plt.ylabel("")

#legend = plt.legend(frameon = 1)

legend = plt.legend(bbox_to_anchor=(0.75, 0.34), title='Type of facility', frameon=1)
frame = legend.get_frame()
frame.set_color('white')
#ax.legend(handles=[])

#plt.savefig('figures/overall_distances_withAK_mercartor.pdf', bbox_inches="tight")
#plt.savefig('figures/pac_divisions.pdf', bbox_inches="tight")

In [ ]:
## see https://coderzcolumn.com/tutorials/data-science/how-to-create-connection-map-chart-in-python-jupyter-notebook-plotly-and-geopandas
params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
plt.rcParams.update(params)

fig, ax = plt.subplots(figsize=(10, 10))
gmap = us_map_div.plot(column= us_map_div['Beds_per_100k'], #color='Children'
            cmap='Greys', ax=ax,  facecolor='white',
           edgecolor='darkgrey', alpha=0.7, linewidth=0.5, aspect=1, 
           legend=True, 
            legend_kwds={'shrink': 1.0, 'label':'Beds per 100k children', 
                         'location':'bottom', 'pad':0.02})
#gmap.set_label('Pediatric population')
#plt.legend(loc=2, prop={'size': 6})
ax.set_facecolor("white")

gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp[~gdf_allstates_stacked_hosp['STATE'].isin(['HI', 'AK'])]

list_status = ['Free-standing pediatric post-acute care facility',
                 'Facility with embedded pediatric post-acute care unit',]
gdf_allstates_stacked_hosp2 = gdf_allstates_stacked_hosp2[gdf_allstates_stacked_hosp2['STATUS_desc'].isin(list_status)]

sns.scatterplot(data=gdf_allstates_stacked_hosp2, x='longitude_2', y='latitude_2', 
                #hue='STATUS_desc', palette = ['Blue', 'DarkOrange', 'DarkRed', 'Black'])
                style='STATUS_desc', color='Black', alpha=0.75)
                #c='Black', s=120, marker='o', 
                #data=gdf_allstates_stacked_hosp)

# ## Loop through each flight plotting line depicting flight between source and destination
# for slat,dlat, slon, dlon in zip(gdf_allstates_stacked['Latitude_Pt_2'], gdf_allstates_stacked["latitude_2"], 
#                                               gdf_allstates_stacked['Longitude_Pt_2'], gdf_allstates_stacked["longitude_2"]):
#     plt.plot([slon , dlon], [slat, dlat], linewidth=1, color="red", alpha=0.35)

#plt.title('(b) Suburban')
plt.xlim(-1.4e7, -0.73e7)#(-130, -65.55)
plt.ylim(0.26e7, 0.65e7)#(24, 55)
plt.xticks([])
plt.yticks([])
plt.xlabel("")
plt.ylabel("")

#legend = plt.legend(frameon = 1)

legend = plt.legend(bbox_to_anchor=(0.75, 0.34), title='Type of facility', frameon=1)
frame = legend.get_frame()
frame.set_color('white')
#ax.legend(handles=[])

#plt.savefig('figures/overall_distances_withAK_mercartor.pdf', bbox_inches="tight")
#plt.savefig('figures/pac_divisions.pdf', bbox_inches="tight")